In [1]:
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
#Load Data

Planet_Testing = spark.read.csv('Planet_Testing.csv',inferSchema=True, header=True)
Planet_Training = spark.read.csv('Planet_Training.csv',inferSchema=True, header=True)

In [4]:
#Select Features

Planet_Testing = Planet_Testing.select('Water', 'Atmosphere Color', 'Temperature', 'Habitable')
Planet_Training  = Planet_Training .select('Water', 'Atmosphere Color', 'Temperature', 'Habitable')

In [5]:
#Data Processing

Planet_Training = Planet_Training.na.drop()
Planet_Testing = Planet_Testing.na.drop()

In [6]:
from pyspark.sql.functions import when

In [7]:
#Transform Data

Planet_Training = Planet_Training.withColumn(
    'Water',
    when(Planet_Training['Water'] == 'Low',0).
    when(Planet_Training['Water'] == 'Medium',1).
    when(Planet_Training['Water'] == 'High',2)
)
Planet_Training = Planet_Training.withColumn(
    'Atmosphere Color',
    when(Planet_Training['Atmosphere Color'] == 'Red',0).
    when(Planet_Training['Atmosphere Color'] == 'Blue',1).
    when(Planet_Training['Atmosphere Color'] == 'Yellow',2)
)
Planet_Testing = Planet_Testing.withColumn(
    'Water',
    when(Planet_Testing['Water'] == 'Low',0).
    when(Planet_Testing['Water'] == 'Medium',1).
    when(Planet_Testing['Water'] == 'High',2)
)
Planet_Testing = Planet_Testing.withColumn(
    'Atmosphere Color',
    when(Planet_Testing['Atmosphere Color'] == 'Red',0).
    when(Planet_Testing['Atmosphere Color'] == 'Blue',1).
    when(Planet_Testing['Atmosphere Color'] == 'Yellow',2)
)

In [8]:
from pyspark.ml.feature import VectorAssembler

In [9]:
assembler = VectorAssembler(
    inputCols = ['Water', 'Atmosphere Color', 'Temperature'],
    outputCol = 'Features'
)

In [10]:
Planet_Training = assembler.transform(Planet_Training)
Planet_Testing = assembler.transform(Planet_Testing)

In [11]:
from pyspark.ml.feature import StandardScaler

In [13]:
#Normalization


scaler = StandardScaler(inputCol = "Features", outputCol = "generated")

Planet_Training = scaler.fit(Planet_Training).transform(Planet_Training)
Planet_Testing = scaler.fit(Planet_Testing).transform(Planet_Testing)

In [14]:
def replace_column(dataset, column_name, temp_column_name):
    dataset = dataset.drop(column_name)
    return dataset.withColumnRenamed(temp_column_name, column_name)

In [15]:
Planet_Training = replace_column(Planet_Training,'Features', 'generated')
Planet_Testing = replace_column(Planet_Testing,'Features', 'generated')

In [16]:
#Generate Model

model = LogisticRegression(
    featuresCol = 'Features',
    labelCol = 'Habitable',
    maxIter = 10
)

In [17]:
model = model.fit(Planet_Training)

In [19]:
#Model Teting and Evaluation

prediction = model.transform(Planet_Testing)
prediction.show()

+-----+----------------+-----------+---------+--------------------+--------------------+--------------------+----------+
|Water|Atmosphere Color|Temperature|Habitable|            Features|       rawPrediction|         probability|prediction|
+-----+----------------+-----------+---------+--------------------+--------------------+--------------------+----------+
|    2|               2|     325145|        1|[2.22498555215050...|[-0.5960950727923...|[0.35523758686965...|       1.0|
|    1|               0|     269079|        0|[1.11249277607525...|[2.99908514599945...|[0.95253277965808...|       0.0|
|    2|               2|     302996|        1|[2.22498555215050...|[-0.2122758672718...|[0.44712941787448...|       1.0|
|    2|               2|     312604|        1|[2.22498555215050...|[-0.3787725455255...|[0.40642297823869...|       1.0|
|    0|               2|     280875|        1|[0.0,2.1827145400...|[-3.0511922662257...|[0.04516602784914...|       1.0|
|    2|               0|     306

In [20]:
evaluator = BinaryClassificationEvaluator(
    rawPredictionCol = 'rawPrediction',
    labelCol = 'Habitable'
)

In [21]:
accuracy = evaluator.evaluate(prediction) * 100
accuracy = round(accuracy, 2)

print(f'Accuracy {accuracy}%')

Accuracy 91.71%
